## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
%tensorflow_version 2.x

In [7]:
import tensorflow as tf
print(tf.__version__)

2.0.0


### Collect Data

In [8]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Residency6_internallab/prices.csv')

### Check all columns in the dataset

In [11]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns=['date','symbol'],inplace=True)

In [13]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data=data.iloc[:1000,:]

In [15]:
data.shape

(1000, 5)

In [0]:
data['volume']=data['volume']/1000000

In [17]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X=data.iloc[:,:-1]
y=data.iloc[:,-1]

In [20]:
X.head()

,open,close,low,high
0,123.430000,125.839996,122.309998,126.250000
1,125.239998,119.980003,119.940002,125.540001
2,116.379997,114.949997,114.930000,119.739998
3,115.480003,116.620003,113.500000,117.440002
4,117.010002,114.970001,114.089996,117.330002


In [21]:
X.shape

(1000, 4)

In [22]:
y.head()

0    2.1636
1    2.3864
2    2.4895
3    2.0063
4    1.4086
Name: volume, dtype: float64

In [23]:
y.shape

(1000,)

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.30,random_state=7)

In [25]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(700, 4)
(700,)
(300, 4)
(300,)


#### Convert Training and Test Data to numpy float32 arrays


In [26]:
X_test.dtypes

open     float64
close    float64
low      float64
high     float64
dtype: object

In [0]:
import numpy as np
X_train=np.array(X_train).astype('float32')
X_test=np.array(X_test).astype('float32')
y_train=np.array(y_train).astype('float32')
y_test=np.array(y_test).astype('float32')

In [28]:
print(X_train.dtype)
print(X_test.dtype)
print(y_train.dtype)
print(y_test.dtype)

float32
float32
float32
float32


In [0]:
#tenserflow needs output variable should always be in column vector. reshape converts it into row vector to column vector. 
y_train=y_train.reshape(-1,1)

In [30]:
y_train.shape

(700, 1)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
norm=Normalizer()

In [0]:
X_train = norm.fit_transform(X_train)
X_test = norm.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias of (Input-hidden Layer) --- 4 features with 5 nueron
w1 = tf.zeros(shape=(4,5))
b1 = tf.zeros(shape=(5))

In [35]:
w1

<tf.Tensor: id=2, shape=(4, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

In [36]:
b1

<tf.Tensor: id=5, shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>

In [0]:
#We are initializing weights and Bias of (hidden - output Layer) --- 4 features with 5 nueron
w2= tf.zeros(shape=(5,1))
b2 = tf.zeros(shape=(1))

In [47]:
w2

<tf.Tensor: id=4082, shape=(5, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>

In [48]:
b2

<tf.Tensor: id=4085, shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):
    y_out = tf.add(tf.matmul(x, w1), b1)
    y_pred = tf.add(tf.matmul(y_out, w2), b2)
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w1, b1, w2, b2, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w1,b1,w2,b2])
        current_prediction = prediction(x, w1, b1, w2, b2)
        current_loss = loss(y_actual, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1, w2, b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    
    return w1, b1,w2,b2

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [53]:
for i in range(100):    
    w1, b1,w2,b2 = train(X_train, y_train, w1, b1,w2,b2)
    if i%5 ==0:
      print('Current Training Loss on iteration', i, loss(y_train, prediction(X_train, w1, b1,w2,b2)).numpy())

('Current Training Loss on iteration', 0, 273.9678)
('Current Training Loss on iteration', 5, 273.95438)
('Current Training Loss on iteration', 10, 273.9543)
('Current Training Loss on iteration', 15, 273.95428)
('Current Training Loss on iteration', 20, 273.95425)
('Current Training Loss on iteration', 25, 273.9542)
('Current Training Loss on iteration', 30, 273.95416)
('Current Training Loss on iteration', 35, 273.95413)
('Current Training Loss on iteration', 40, 273.95407)
('Current Training Loss on iteration', 45, 273.954)
('Current Training Loss on iteration', 50, 273.95398)
('Current Training Loss on iteration', 55, 273.95392)
('Current Training Loss on iteration', 60, 273.9539)
('Current Training Loss on iteration', 65, 273.95383)
('Current Training Loss on iteration', 70, 273.9538)
('Current Training Loss on iteration', 75, 273.95374)
('Current Training Loss on iteration', 80, 273.95367)
('Current Training Loss on iteration', 85, 273.95367)
('Current Training Loss on iteration'

### Get the shapes and values of W and b

In [54]:
w1.numpy()

array([[0.761003  , 0.761003  , 0.761003  , 0.761003  , 0.761003  ],
       [0.78630155, 0.78630155, 0.78630155, 0.78630155, 0.78630155],
       [0.75765836, 0.75765836, 0.75765836, 0.75765836, 0.75765836],
       [0.7934293 , 0.7934293 , 0.7934293 , 0.7934293 , 0.7934293 ]],
      dtype=float32)

In [55]:
b1.numpy()

array([1.549563, 1.549563, 1.549563, 1.549563, 1.549563], dtype=float32)

In [56]:
w2.numpy()

array([[0.3546951],
       [0.3546951],
       [0.3546951],
       [0.3546951],
       [0.3546951]], dtype=float32)

In [57]:
b2.numpy()

array([0.11448701], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

In [58]:
pred=prediction(X_test[:1,:],w1,b1,w2,b2)  ## for test data we only apply foerard pass only. 
pred

<tf.Tensor: id=9441, shape=(1, 1), dtype=float32, numpy=array([[5.610027]], dtype=float32)>

In [59]:
print('Current Testing Loss on 1st Example', loss(y_test, prediction(X_test[:1,:],w1,b1,w2,b2)).numpy())

('Current Testing Loss on 1st Example', 57.884544)


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Residency6_internallab/11_Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [62]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
iris_encoded=pd.get_dummies(iris,prefix=['Species'])

In [67]:
iris_encoded.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
X=iris_encoded.iloc[:,:-3]

In [69]:
X.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,1,5.1,3.5,1.4,0.2
1,2,4.9,3.0,1.4,0.2
2,3,4.7,3.2,1.3,0.2
3,4,4.6,3.1,1.5,0.2
4,5,5.0,3.6,1.4,0.2


In [0]:
y=iris_encoded.iloc[:,-3:]

In [71]:
y.head()

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [0]:
X=X.values

In [0]:
y=y.values

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# Initialize Sequential model
model = tf.keras.models.Sequential()

# Normalize the data
model.add(tf.keras.layers.BatchNormalization())

# Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, input_shape=(4,), activation='softmax'))

In [0]:
# Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training 

In [80]:
model.fit(X,y, epochs=100,
          batch_size = X.shape[0])

Train on 150 samples
Epoch 1/100
150/150 [==============================] - 0s 61us/sample - loss: 0.7778 - accuracy: 0.6667
Epoch 2/100
150/150 [==============================] - 0s 37us/sample - loss: 0.7727 - accuracy: 0.6667
Epoch 3/100
150/150 [==============================] - 0s 26us/sample - loss: 0.7677 - accuracy: 0.6667
Epoch 4/100
150/150 [==============================] - 0s 28us/sample - loss: 0.7628 - accuracy: 0.6733
Epoch 5/100
150/150 [==============================] - 0s 31us/sample - loss: 0.7580 - accuracy: 0.6733
Epoch 6/100
150/150 [==============================] - 0s 26us/sample - loss: 0.7533 - accuracy: 0.6800
Epoch 7/100
150/150 [==============================] - 0s 20us/sample - loss: 0.7486 - accuracy: 0.6933
Epoch 8/100
150/150 [==============================] - 0s 32us/sample - loss: 0.7441 - accuracy: 0.7000
Epoch 9/100
150/150 [==============================] - 0s 26us/sample - loss: 0.7396 - accuracy: 0.7067
Epoch 10/100
150/150 [=====================

### Model Prediction

In [83]:
model.predict(X,batch_size=X.shape[0])

array([[0.78499854, 0.18064202, 0.03435946],
       [0.70962715, 0.24371615, 0.04665675],
       [0.7851802 , 0.17725205, 0.03756767],
       [0.78402954, 0.17580193, 0.04016854],
       [0.8051385 , 0.16052826, 0.03433321],
       [0.7993704 , 0.16338846, 0.03724113],
       [0.8252056 , 0.13898225, 0.03581207],
       [0.75521433, 0.20038792, 0.04439778],
       [0.75457364, 0.19591564, 0.04951067],
       [0.6957904 , 0.2500555 , 0.05415408],
       [0.728757  , 0.22390968, 0.0473333 ],
       [0.77715033, 0.17698646, 0.04586327],
       [0.67367023, 0.26587737, 0.06045235],
       [0.76323   , 0.18775176, 0.04901822],
       [0.68525594, 0.26376516, 0.05097885],
       [0.79856145, 0.1610343 , 0.04040416],
       [0.7404843 , 0.20732018, 0.05219548],
       [0.70662606, 0.23262447, 0.06074944],
       [0.6526626 , 0.2805646 , 0.06677276],
       [0.76717913, 0.18062696, 0.05219395],
       [0.6038751 , 0.31791237, 0.07821254],
       [0.73625654, 0.20191494, 0.06182854],
       [0.

In [85]:
    score = model.evaluate( X,y,batch_size=X.shape[0])
    score

150/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.5384694337844849, 0.7733333]

In [87]:
model.metrics_names

['loss', 'accuracy']

### Save the Model

In [88]:
model.save

<bound method Sequential.save of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f56b3971e10>>

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [102]:
input_points = 4
hidden_layers = 10
output_nodes=3

# Initialize Sequential model
model2 = tf.keras.models.Sequential()

# Normalize the data
model2.add(tf.keras.layers.BatchNormalization())

# Add Dense Layer which provides hidden layers after applying softmax for hidden layer
model2.add(tf.keras.layers.Dense(hidden_layers, input_dim=input_points, kernel_initializer='normal', activation='softmax'))

# hidden to next hidden layer 
model2.add(tf.keras.layers.Dense(hidden_layers, activation='softmax'))

# next-hidden to output layer 
model2.add(tf.keras.layers.Dense(output_nodes, activation='softmax'))

sgd = tf.keras.optimizers.SGD(lr=.01, decay=1e-6, momentum=0.9)

# Comile the model
model2.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

model2.fit(X,y, epochs=100,
          batch_size = X.shape[0])

score2 = model2.evaluate( X,y,batch_size=X.shape[0])
print('Score: ',score2)
print('Metrics_names: ', model2.metrics_names)

Train on 150 samples
Epoch 1/100
150/150 [==============================] - 0s 2ms/sample - loss: 1.0989 - accuracy: 0.3333
Epoch 2/100
150/150 [==============================] - 0s 37us/sample - loss: 1.0989 - accuracy: 0.3333
Epoch 3/100
150/150 [==============================] - 0s 24us/sample - loss: 1.0989 - accuracy: 0.3333
Epoch 4/100
150/150 [==============================] - 0s 24us/sample - loss: 1.0989 - accuracy: 0.3333
Epoch 5/100
150/150 [==============================] - 0s 26us/sample - loss: 1.0989 - accuracy: 0.3333
Epoch 6/100
150/150 [==============================] - 0s 25us/sample - loss: 1.0989 - accuracy: 0.3333
Epoch 7/100
150/150 [==============================] - 0s 24us/sample - loss: 1.0989 - accuracy: 0.3333
Epoch 8/100
150/150 [==============================] - 0s 23us/sample - loss: 1.0989 - accuracy: 0.3333
Epoch 9/100
150/150 [==============================] - 0s 22us/sample - loss: 1.0989 - accuracy: 0.3333
Epoch 10/100
150/150 [======================

After adding 2 hidden layers, I see the performance decreased. Tried to optimize it but still it is giving the same performance.